In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from Configuration import printPeriodic,setDbgPrint,null
from data_Generator import data_Generator,data_Generator_label
import io, os, sys, types
from Utils import NotebookFinder
sys.meta_path.append(NotebookFinder())

In [ ]:
class runTest():
    def __init__(self ):
        self.actions = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14]
        self.n_actions = len(self.actions)
        self.n_features = 76
        self.built_net()
        self.TPcount = 0
        self.FPcount = 0
        self.TNcount =0
        self.FNcount = 0
        # sess = tf.InteractiveSession()
        # self.saver.restore(sess, 'my_net/my_test_model.ckpt-20000')  # Load parameter
        self.params = []
        self.cost_his = []
        self.time = 0
        self.flag = 0
        self.list = data_Generator(self.flag)
        self.label = data_Generator_label(self.flag)
        self.reward_history = []
        self.count_history =[]
        self.Pr_history = []
        self.Rc_history = []
        self.time_env_state = {'current': self.data()}
        target_net = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='target_net')
        saver1 = tf.train.Saver(target_net)
        self.sess = tf.Session()
        #Upload model parameters
        saver1.restore(self.sess, "./my_net/target_net.ckpt-206000")

    def built_net(self):
        tf.reset_default_graph()
        # Reconstruction neural network model.Only one of the neural networks is kept here.
       # c_names, n_l1, w_initializer, b_initializer = ['target_net_params', tf.GraphKeys.GLOBAL_VARIABLES], 10, tf.random_normal_initializer(0., 0.3), tf.constant_initializer(0.1)
        n_l1 = 10
        self.env_state_ = tf.placeholder(tf.float32, [None, self.n_features], name='env_state_')
        with tf.variable_scope('target_net'):
            c_names = ['target_net_params', tf.GraphKeys.GLOBAL_VARIABLES]

            with tf.variable_scope(
                    'l1'):
                w1 = tf.get_variable('w1', [self.n_features, n_l1], collections=c_names)
                b1 = tf.get_variable('b1', [1, n_l1], collections=c_names)
                l1 = tf.nn.tanh(tf.matmul(self.env_state_, w1) + b1)
            with tf.variable_scope('l2'):
                w2 = tf.get_variable('w2', [n_l1, self.n_actions], collections=c_names)
                b2 = tf.get_variable('b2', [1, self.n_actions], collections=c_names)
                self.q_next = tf.matmul(l1, w2) + b2


    def choose_action(self, env_state):
        env_state = env_state[np.newaxis, :]
        actions_value = self.sess.run(self.q_next,feed_dict={self.env_state_: env_state})
        action = np.argmax(actions_value)
        return action

    def data(self):
        Network_data = self.list[self.time]
        return Network_data

    def update_State(self):
        self.time_env_state["next"] = self.data()
        self.time += 1
        return self.time_env_state["next"]

    def reset(self):
        self.update_State()
        return self.time_env_state["next"]

    def reward(self, action):

        if action == self.label[self.time - 1][0]:
            reward = 10
        else:
            reward = 0

        return reward

    def step(self, action):

        value = 0
        min_value = np.Inf
        action_key = ""

        #self.log('{}: action {} has min. value {}\n'.format(self.time, action_key, min_value), period=self.statusPeriod,
        #         counter=self.time)
        # next_state = action
        reward = self.reward(action)
        self.reward_ = reward

        self.count_history.append(self.reward_)
        # self.correct_rate.append(self.count/(len(self.count_history)))

        self.state = self.update_State()

        return self.time_env_state["current"], self.time_env_state["next"], reward, self.label[self.time - 2][0]

    def run_(self,cfg=None):
        numEpisodes = cfg['numEpisodesTest']
        maxSteps = cfg['maxStepsTest']
        dbgPrint = cfg.get('dbgPrint', null)

        statusPeriod = cfg.get('statusPeriod', 1)
        for episode in range(1,numEpisodes+1):
            # initial observation
            step = 0
            env_state_1 = self.reset()
            while step < maxSteps:
                #print('{}: current env = {}\n'.format(self.time, self.time_env_state))
                env_state = np.hstack(env_state_1)

                action = self.choose_action(env_state)
               # print('{}: action_ = {}, observation = {}\n'.format(self.time, action_, observation))
                env_state_, observation_,reward,label = self.step(action)
                if label != 0 and action != 0:
                    self.TPcount += 1
                elif action != 0 and label == 0:
                    self.FPcount += 1
                elif action == 0 and label == 0:
                    self.TNcount += 1
                elif action == 0 and label != 0:
                    self.FNcount += 1
                if ((self.TPcount + self.FNcount) != 0 and (self.TPcount + self.FPcount) != 0):
                    Pr = self.TPcount / (self.TPcount + self.FPcount)
                    Rc = self.TPcount / (self.TPcount + self.FNcount)
                    F1 = 2 / (1 / Pr + 1 / Rc)
                    self.Rc_history.append(self.TPcount/(self.TPcount+self.FNcount))
                    self.Pr_history.append(self.TPcount/(self.TPcount+self.FPcount))
                    print('Rc = {},Pr = {},F1 = {}'.format(Rc,Pr,F1))

                #print("env_state = {},\nlabel={}, action = {},reward = {},\nobservation_ ={}\n".format(env_state_, label,
                 #                                                                                 action, reward,
                  #                                                                                 observation_))
                env_state_ = np.hstack(observation_)
                env_state_1 = env_state_
                step += 1